# 분류와 구조화된 출력
## 텍스트를 라벨로 분류하기

- LangChain과 Pydantic을 활용하여 텍스트 분류(Classification) 태스크를 수행

**태깅(Tagging)** 이란 문서에 다음과 같은 클라스를 지정하는 것을 의미합니다.

- 감정 (sentiment)
- 언어
- 스타일(공식적, 비공식적 등)
- 다루는 주제
- 정치적 경향


## **개요 (Overview)**  

**태깅(Tagging)** 은 몇 가지 주요 구성 요소로 이루어집니다:  

- **`function`**: 추출(Extraction)과 마찬가지로, 태깅은 함수(Functions)를 사용하여 모델이 문서를 어떻게 태깅해야 하는지 명시합니다.  
- **`schema`**: 문서를 어떻게 태깅할지 정의합니다 --> Pydantic 데이터 모델을 이용하여 정의

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

## Pydantic 이란? 
- Pydantic은 Python 데이터 검증 및 설정 관리를 위한 라이브러리
- Pydantic의 핵심 기능
    - 타입 검증 (Type Validation)
    - 데이터가 지정된 형식(int, str 등)과 일치하는지 자동 확인
    - 기본값 설정 (Default Values)

In [13]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str
    is_active: bool = True  # 기본값 설정 가능

user = User(id=123, name='aaa', email='bbb', is_active=True)
user

User(id=123, name='aaa', email='bbb', is_active=True)

In [18]:
print(user)
print(user.id, user.name, user.email, user.is_active)

id=123 name='aaa' email='bbb' is_active=True
123 aaa bbb True


In [19]:
from pydantic import BaseModel, Field, ValidationError

class Classification(BaseModel):
    agressiveness: int = Field(
        default=1, 
        description="공격성 수치 (1~10 사이의 값)",
        ge=1, le=10  # 최소값 1, 최대값 10 제한
    )

# 올바른 데이터 (1~10 범위 내)
valid_data = Classification(agressiveness=5)
print(valid_data)

# 잘못된 데이터 (10 초과)
try:
    invalid_data = Classification(agressiveness=15)  # 10보다 큼
except ValidationError as e:
    print("\n❌ 유효성 검사 오류 발생:")
    print(e)

agressiveness=5

❌ 유효성 검사 오류 발생:
1 validation error for Classification
agressiveness
  Input should be less than or equal to 10 [type=less_than_equal, input_value=15, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


### Pydantic을 이용하여 LLM의 구조화된 출력 유도

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    이름: str
    날자: str
    참석자: list[str]

calendar_prompt = ChatPromptTemplate.from_template(

"""
다음 글에서 원하는 정보를 추출하세요.
'CalendarEvent' 에 언급된 속성만 추출하세요.

글:
{input}
"""
)

# Structured Output(구조화된 출력) 생성
llm = model.with_structured_output(CalendarEvent)

inp = "Alice와 Bob은 금요일에 과학 박람회에 갈 예정입니다."
prompt = calendar_prompt.invoke({"input": inp})
llm.invoke(prompt)

CalendarEvent(이름='과학 박람회', 날자='금요일', 참석자=['Alice', 'Bob'])

### 구조화된 출력값의 범위 조절

In [21]:
class Classification_2(BaseModel):
    sentiment: str = Field(..., enum=["행복하다", "중립적", "슬프다"])  # ... - 필수 입력을 의미
    aggressiveness: int = Field(
        ...,   # 필수 입력 필드임
        description="문장이 얼마나 공격적인지를 나타내며 숫자가 높을수록 더 공격적입니다.",
        enum=[1, 2, 3, 4, 5],  # 1~5 값만 허용
    )

# 올바른 데이터 입력
valid_data = Classification_2(sentiment="행복하다", aggressiveness=3)
print(valid_data)

# 필수 필드 누락 (오류 발생)
try:
    invalid_data = Classification_2(sentiment="슬프다")  # aggressiveness 값 없음!
except ValidationError as e:
    print("\n❌ 유효성 검사 오류 발생:")
    print(e)

sentiment='행복하다' aggressiveness=3

❌ 유효성 검사 오류 발생:
1 validation error for Classification_2
aggressiveness
  Field required [type=missing, input_value={'sentiment': '슬프다'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


## Schema 정의와 도구 호출 

LangChain에서 OpenAI의 **도구 호출(Tool Calling)** 기능을 사용하여 태깅을 수행하는 간단한 예제를 살펴보겠습니다.  

- OpenAI 모델에서 지원하는 `with_structured_output` 메서드를 사용할 것입니다.  

스키마에 몇 가지 속성과 예상 유형을 추가하여 Pydantic 모델을 지정해 보겠습니다.

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 프롬프트 템플릿 정의
# 주어진 텍스트에서 필요한 정보를 추출하도록 지침 제공
tagging_prompt = ChatPromptTemplate.from_template(
"""
다음 글에서 원하는 정보를 추출하세요.
'Classification' 함수에 언급된 속성만 추출하세요.

글:
{input}
"""
)

# Pydantic 데이터 모델을 이용하여 텍스트에서 추출할 속성 정의
# 감정의 종류와 값의 범위를 자율 지정
class Classification(BaseModel):
    sentiment: str = Field(description="텍스트의 감정")
    agressiveness: int = Field(
        description="텍스트가 1~10점 척도로 얼마나 공격적인지를 나타냅니다."
    )

# Structured Output(구조화된 출력) 생성
llm = model.with_structured_output(Classification)
llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10d15b410>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10d269010>, root_client=<openai.OpenAI object at 0x10c9d7bd0>, root_async_client=<openai.AsyncOpenAI object at 0x10d268b50>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Classification'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'description': '텍스트의 감정', 'type': 'string'}, 'agressiveness': {'description': '텍스트가 1~10점 척도로 얼마나 공격적인지를 나타냅니다.', 'type': 'integer'}}, 'required': ['sentiment', 'agressiveness'], 'type': 'object'}}}}}, config={}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(...), kwargs={}, config={},

In [25]:
inp = "너를 만나게 되어 정말 기뻐! 우리는 아주 좋은 친구가 될 것 같아!"

prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', agressiveness=1)

dictionary 출력을 원하면 `.model_dump()`를 호출하면 됩니다.

In [26]:
response.model_dump()

{'sentiment': 'positive', 'agressiveness': 1}

예제에서 볼 수 있듯이, 모델은 우리가 원하는 바를 정확하게 해석합니다.  

다음 섹션에서는 이러한 결과를 어떻게 제어할 수 있는지 살펴보겠습니다.

## **더 세밀한 출력 제어**

**스키마(schema)** 를 더 자세히 정의하면 모델의 출력을 더 세밀하게 제어할 수 있습니다.  

구체적으로 다음을 정의할 수 있습니다:  

- **각 속성의 가능한 값**  
- **속성을 모델이 정확하게 이해할 수 있도록 설명 추가**  
- **반드시 반환해야 할 필수 속성**  

이전에 언급한 각 요소를 제어하기 위해 **Enums**를 사용하여 우리의 **Pydantic 모델**을 다시 선언해봅시다.



In [27]:
model = init_chat_model("gpt-5-mini", model_provider="openai")

In [28]:
# 필수 요소로 지정하고 감정의 종류와 값의 범위 지정
class Classification_2(BaseModel):
    sentiment: str = Field(..., enum=["행복하다", "화난다", "슬프다"])
    aggressiveness: int = Field(
        ...,
        description="문장이 얼마나 공격적인지를 나타내며 숫자가 높을수록 더 공격적입니다.",
        enum=[1, 2, 3, 4, 5],
    )

llm = model.with_structured_output(Classification_2)

이제 답변은 우리가 예상하는 방식으로 제한될 것입니다!

In [34]:
inps = ["너를 만나면 죽여 버릭거야! 우리는 영원히 친구가 될 수 없어!",
        "너를 보면 눈물이 나!",
        "여기 날씨는 괜찮아요, 코트 하나 없이도 밖에 나갈 수 있어요"]

for inp in inps:
    prompt = tagging_prompt.invoke({"input": inp})
    response = llm.invoke(prompt)
    print(response.model_dump())

{'sentiment': '화난다', 'aggressiveness': 5}
{'sentiment': '슬프다', 'aggressiveness': 1}
{'sentiment': '행복하다', 'aggressiveness': 1}


# 구조화된 출력을 활용한 콘텐츠 심사

In [38]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel

class Category(str, Enum):
    violence = "폭력"
    sexual = "성적인내용"
    self_harm = "자해"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

llm = model.with_structured_output(ContentCompliance)

tagging_prompt = ChatPromptTemplate.from_template(
"""
사용자 입력이 특정 지침을 위반하는지 판단하고 'ContentCompliance' 형식에 맞추어 출력하세요.
한국어로 출력하세요.

글:
{input}
"""
)

inps = ["요즘 너무 힘들고 삶을 포기하고 싶어.",
        "이 사진은 성적으로 노골적인 내용을 포함하고 있어.",
        "니 대가리를 박살낼거야"]

for inp in inps:
    prompt = tagging_prompt.invoke({"input": inp})
    response = llm.invoke(prompt)
    print(response.model_dump())
    print()

{'is_violating': True, 'category': <Category.self_harm: '자해'>, 'explanation_if_violating': "사용자가 '삶을 포기하고 싶다'고 명시하여 자해·자살 의사를 표현하고 있습니다. 이는 자해 관련 지침(자해 고취 금지 및 안전 조치 필요)에 해당하므로 위반으로 판단됩니다."}

{'is_violating': False, 'category': None, 'explanation_if_violating': None}

{'is_violating': True, 'category': <Category.violence: '폭력'>, 'explanation_if_violating': '직접적인 폭력 위협 문장입니다. 타인의 신체에 해를 가하겠다는 명백한 위협으로 폭력 관련 정책을 위반합니다.'}



# 실습 : 구조화된 출력 코드 작성

### 문제 1: 캘린더 이벤트 추출 (기초)

아래 문장에서 **이름**, **날짜**, **참석자** 정보를 추출하도록 `CalendarEvent` 클래스를 정의하고, .

```text
"민수, 유리, 그리고 지민은 다음 주 화요일에 회사 워크숍에 참석합니다."
```

<details>
<summary>힌트</summary>

```
class CalendarEvent(BaseModel):
    이름: str
    날자: str
    참석자: list[str]
```

</details>